In [ ]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics

In [ ]:
MODEL = "yolov8x.pt"
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [00:00<00:00, 223MB/s]


YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [ ]:
!yolo task=detect mode=predict model=yolov8x.pt conf=0.25 source={"/content/stret1.mp4"}


Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 568, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 439, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/predictor.py", line 177, in predict_cli
    for _ in gen:  # noqa, running CLI inference without accumulating any outputs (do not modify)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 35, in generator_context
    response = gen.send(None

In [ ]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - car, motorcycle, bus and truck
CLASS_ID = [0,1,2, 3, 5, 7,9]

In [ ]:
!pip install supervision


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.18.0


In [ ]:
import supervision as sv
import numpy as np
from ultralytics import YOLO
import cv2

# Define the model and tracker
MODEL = "yolov8x.pt"
model = YOLO(MODEL)
tracker = sv.ByteTrack()


#Defining line co-ordinates for counting objects crossing the line
LINE_START = sv.Point(20, 400)
LINE_END = sv.Point(1080, 400)


#Initialize Line Annotator with custom thickness and font size
line_zone = sv.LineZone(start=LINE_START , end=LINE_END)
line_annotator = sv.LineZoneAnnotator(thickness=1, text_thickness=2, text_scale=2)

# Define the callback function
def callback(frame: np.ndarray, index: int) -> np.ndarray:

    # Perform object detection
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)

    # Update tracker with detections
    detections = tracker.update_with_detections(detections)


    # Extract labels for annotations
    labels = []
    for tracker_id, class_id in zip(detections.tracker_id, detections.class_id):
        class_name = CLASS_NAMES_DICT[class_id]
        labels.append(f"#{tracker_id} - {class_name}")



    #Draw the Line
    line_annotator.annotate(line_zone)

    #Trigger the Line Annotation for Updating Counter
    line_zone.trigger(detections=detections)


# Process the video
sv.process_video(
    source_path="/content/street1.mp4",
    target_path="/content/final_result.mp4",
    callback=callback
)


0: 640x512 15 cars, 1 truck, 1 clock, 3618.2ms
Speed: 5.4ms preprocess, 3618.2ms inference, 7.9ms postprocess per image at shape (1, 3, 640, 512)


TypeError: LineZoneAnnotator.annotate() missing 1 required positional argument: 'line_counter'

In [ ]:
video_info = sv.VideoInfo.from_video_path(video_path='/content/final_result.mp4')
video_info

VideoInfo(width=672, height=884, fps=60, total_frames=391)